# Tokenisation

La tokenisation est le processus de segmentation d’un texte en unités (tokens) significatives (phrases, mots, symboles…). Elle constitue souvent la première étape avant toute analyse de texte.

Commençons par charger la bibliothèque spaCy qui facilite les opérations de segmentation, ainsi que deux textes en anglais et en français :

In [ ]:
import spacy

en = "Little Joe sat down on the bank and prepared to enjoy his breakfast. He hadn't seen Buster Bear, and he didn't know that he or any one else was anywhere near."
fr = "Elvire, m’as-tu fait un rapport bien sincère ? Ne déguises-tu rien de ce qu’a dit mon père ?"

nlp_en = spacy.blank("en")
nlp_fr = spacy.blank("fr")

## Segmentation en phrases

La segmentation en phrases n’est pas disponible par défaut dans les modèles de langage de spaCy. La méthode `.add_pipe()` permet de charger la ressource nécessaire dans le pipeline :

In [ ]:
# add sentencizer to the pipeline
nlp_en.add_pipe('sentencizer')
nlp_fr.add_pipe('sentencizer')

La liste des composants installés dans un pipeline est ensuite accessible via une liste stockée dans un attribut `.components` :

In [ ]:
nlp_en.components

Nous pouvons ensuite passer les textes dans les pipelines :

In [ ]:
# process the documents
doc_en = nlp_en(en)
doc_fr = nlp_fr(fr)

print(
    [ sent.text for sent in doc_en.sents ],
    [ sent.text for sent in doc_fr.sents ],
    sep="\n"
)

## Segmentation en mots

Cette opération est inclue nativement dans les pipelines de spaCy :

In [ ]:
for token in doc_en:
    print(token.text, end=" ")

Parmi les choix notables, elle conserve les signes de ponctuation et sépare les formes contractées : *hadn't* devient *had* et *n't*. Regardons si les choix sont identiques pour le français :

In [ ]:
for token in doc_fr:
    print(token.text, end=" ")

La segmentation est conforme à ce que l’on voudrait pour du français : *m'as-tu* a bien été divisé en quatre tokens avec, pour l’apostrophe accrochée au premier d’entre eux.

### Une segmentation sur les caractères blancs

Comment personnaliser le tokenisateur ? Si l’objectif est de conserver les contractions de l’anglais ou les élisions du français, il est possible de programmer facilement un composant qui fasse la séparation sur les caractères blancs :

In [ ]:
from spacy.tokenizer import Tokenizer

tokenizer_en = Tokenizer(nlp_en.vocab)
tokenizer_fr = Tokenizer(nlp_fr.vocab)

print(
    tokenizer_en(en),
    tokenizer_fr(fr),
    sep="\n"
)

L’idée est d’entraîner une instance de la classe `Tokenizer` à l’aide du seul vocabulaire du modèle de langage concerné. Ces vocabulaires contiennent des informations statistiques sur les mots-formes des langues traitées.

### Ajouter des règles spéciales

La plupart du temps, il sera plutôt question d’agir sur le tokenisateur par défaut en lui ajoutant des règles spéciales. Mais avant, cela implique de bien comprendre les étapes de la tokenisation :

![Étapes de la tokenisation](./images/tokenization.svg)

Le schéma peut se comprendre ainsi :

1. Segmenter grâce aux espaces ;
2. vérifier si une règle spéciale s’applique à chaque sous-chaîne ;
3. rechercher une correspondance avec un token ;
4. autrement rechercher un préfixe ;
5. en l’absence de préfixe, rechercher un suffixe ;
6. en l’absence de suffixe, vérifier à nouveu si une règle spéciale ne s’applique pas.

À ces étapes, il faudrait aussi appliquer quelques autres règles, comme la recherche des infixes (traits d’unions…) et des motifs d’URL.

Regardons une phrase en français qui ne devrait guère poser de difficultés :

In [ ]:
doc = nlp_fr("Y a-t-il un médecin dans la salle ?")
for t in doc:
    print(t.text)

Lorsque, plus tôt, *spaCy* a bien analysé l’inversion du sujet avec la structure *m'as-tu*, il échoue à la repérer ici. Pour preuve, le trait d’union reste attaché aux tokens *-t* et *-il*.

Pour remédier à ce défaut, ajoutons une règle spéciale au tokenisateur grâce à la méthode `.add_special_case()` :

In [ ]:
from spacy.symbols import ORTH

special_case = [{ORTH: "a"}, {ORTH: "-"}, {ORTH: "t"}, {ORTH: "-"}, {ORTH: "il"}]

nlp_fr.tokenizer.add_special_case("a-t-il", special_case)

doc = nlp_fr("Y a-t-il un médecin dans la salle ?")
for token in doc:
    print(token.text)

## Filtrer une liste de tokens

Après la phase de segmentation, il arrive souvent que l’on ne veuille pas conserver tous les tokens identifiés dans un texte.

### Supprimer les mots vides

L‘une des opérations les plus courantes consiste à filtrer les mots vides d’une langue pour ne retenir que ceux signifiants. Chaque modèle de langage dispose d’une propriété `.stop_words` pour les lister :

In [ ]:
stopwords_fr = spacy.lang.fr.stop_words.STOP_WORDS

Le filtrage peut s’effectuer de manière procédurale :

In [ ]:
filtered = [
    token
    for token in doc_fr
    if token.text.lower() not in stopwords_fr
]

Ou, mieux, grâce à la fonction `filter()` :

In [ ]:
filtered = filter(lambda token: token.text.lower() not in stopwords_fr, doc_fr)

Observons le résultat du traitement :

In [ ]:
for token in filtered:
    print(token.text, end=" ")

### Supprimer les signes de ponctuation

En soi, il n’est pas nécessaire avec *spaCy* de supprimer les signes de ponctuation. L’attribut lexical `.is_punct` permet de les exclure d’un traitement :

In [ ]:
for token in doc_fr:
    if not token.is_punct:
        print(token.text, end=" ")

Autrement, il est facile de l’intégrer au filtre :

In [ ]:
filtered = filter(lambda token: token.text.lower() not in stopwords_fr and not token.is_punct, doc_fr)
for token in filtered:
    print(token, end=" ")

### Autres traitements

Pour rappel, il existe deux autres attributs lexicaux qui peuvent être utilisés afin de filtrer les tokens :

- `.is_alpha` : le token est-il constitué de caractères alphabétiques ?
- `.is_space` : le token fait-il partie de la familles des caractères blancs ?
- `.like_num` : le token ressemble-t-il à un nombre ?

In [ ]:
for token in nlp_fr("En 1492, Christophe Colomb a semble-t-il découvert l’Amérique."):
    if not token.like_num and not token.is_punct:
        print(token)